# Imports

Import the required libraries.

In [ ]:
import tensorflow as tf

import get_data
import get_model

In [ ]:
train_dataset = get_data.get_dataset(
    "./data/next_day_wildfire_spread_train*",
    data_size=64,
    sample_size=64,
    batch_size=100,
    num_in_channels=2,
    compression_type=None,
    include_frp=True
)

val_dataset = get_data.get_dataset(
    "./data/next_day_wildfire_spread_eval*",
    data_size=64,
    sample_size=64,
    batch_size=100,
    num_in_channels=2,
    compression_type=None,
    include_frp=True
)

In [ ]:
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
model = get_model.get_model((64, 64, 3))

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001, decay_steps=15, decay_rate=0.96, staircase=True
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=dice_coef_loss,
    metrics=[
        dice_coef,
        tf.keras.metrics.AUC(curve="PR"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

In [ ]:
model.fit(
    train_dataset, validation_data=val_dataset, epochs=10,
    callbacks=[tf.keras.callbacks.ModelCheckpoint(
        "./model_best.h5",
        save_weights_only=True,
        save_best_only=True
    )]
)